In [2]:
import json
import pandas as pd
from datetime import datetime

In [3]:
LIST_OF_CHANNEL_IDS = [
"UCsDmESjqNPukDmVnuneLrqw",
"UCY6Ij8zOds0WJEeqCLOnqOQ",
"UClN24S5-fc7LD0JJW0FdGWw",
"UC3iNdSV_RQU7DHTGpqEW96w",
"UC_52i6mlrzOZs_NEuSepzAg",
"UCr3cBLTYmIK9kY0F_OdFWFQ",
"UCV6g95OBbVtFmN9uiJzkFqQ",
"UCgCKYs56-LKEPGQ99DzqQOg",
"UCOwxx9VnEnlFKt5EB70KTzQ",
"UC2Ds30pkifFVD0CE08wF50g",
"UC7UGbBVrqLlq6CRxWWUmyKw",
"UCn1XB-jvmd9fXMzhiA6IR0w",
"UCFeqAfEuKm7lIg2ddQzh61A",
"UCAlxwHEm1bWWYoe5VCSQYEg",
"UCz7iJPVTBGX6DNO1RNI2Fcg",
"UCOAUz4d1NUNxSHh_SOJtdGQ",
"UCGGTAB19HlHEWPwwmxHsEKA",
"UCOsATJw-IZgqGT8MFrHjKGg",
"UCsKVP_4zQ877TEiH_Ih5yDQ",
"UCOinp9hALPUcZeoL3Kmaviw",
"UCGwu0nbY2wSkW8N-cghnLpA",
"UCu6v4AdYxVH5fhfq9mi5llA",
"UCj74rJ9Lgl3WTngq675wxKg",
"UCNIuvl7V8zACPpTmmNIqP2A",
"UCo8bcnLyZH8tBIH9V1mLgqQ",
"UCR0O-1cvuPNxDosvSDLpWHg",
"UCvUmwreRrbxeR1mbmojj8fg",
"UCUKi4zY5ETSqrKAjTBgjM-g",
"UCDQBZcjYKP1J1Nu-Y0_D37Q",
"UCQ9HvHH-KRYHI5ynj2kbLwQ",
"UCRfg0SWjIHm_h95e4V8X5og",
"UCWPB0WpnMIy-g7zncwIhvQg",
"UCC-slLJZ4p4HOznMUcFn_2g"]

So I get the list of videos that a channel has. I don't yet have the specifics (aka metadata) on the videos yet because this API does not give me the information. What I have to do is filter this information to get a list of videos I need to throw at a different API to get the metadata. 

In [4]:
df = pd.read_json (r'searchListResponse.json')
channelIdToListOfVids = {} #channelId: []vids
backend_purposes = {}

#initialize the dicts
for channelId in LIST_OF_CHANNEL_IDS:
    channelIdToListOfVids[channelId] = []
    backend_purposes[channelId] = []

#youtuber
#0~9ish: items
#items: 0~46ish
test = df.loc[0, LIST_OF_CHANNEL_IDS[0]]
for index, midList in df.iterrows():  
    listOfLists = df.loc[index, LIST_OF_CHANNEL_IDS[index]]
    for indList in listOfLists:
        for item in indList['items']:
            channelId = item['snippet']['channelId']
            if 'videoId' in item['id'].keys():
                snippet=item['snippet']
                vidId = item['id']['videoId']
                publishTime = snippet['publishTime']
                vidTitle = snippet['title']
                vidDesc = snippet['description']

                vidInfo = {'vidId' : vidId,
                'publishTime': publishTime,
                'vidTitle': vidTitle,
                'vidDescription': vidDesc}
                backend_purposes[channelId].append(vidId)
                channelIdToListOfVids[channelId].append(vidInfo)

count= 0
for i in backend_purposes.keys():
    count += len(backend_purposes[i])
print(count, " videos detected")

print(len(channelIdToListOfVids.keys()), " keys detected")
vidCont=0
for i in channelIdToListOfVids.keys():
    vidCont+=len(channelIdToListOfVids[i])
print(count, " videos detected")
channelIdToListOfVids

3223  videos detected
33  keys detected
3223  videos detected


{'UCsDmESjqNPukDmVnuneLrqw': [{'vidId': 'beSntf_CNKA',
   'publishTime': '2019-09-18T21:07:40Z',
   'vidTitle': 'Mean Girls Got Me A Wife (Part 3)',
   'vidDescription': "Mean Girls is a great movie.... and my wife agreed. Huion Contest: http://www.itsalexclark.com/huion Pam's channel: ..."},
  {'vidId': '9p763a_DGHo',
   'publishTime': '2016-09-21T12:12:34Z',
   'vidTitle': '&quot;Bonding&quot; With Uncle',
   'vidDescription': "Uncle thinks he's the only one with a gun... Now you can mail me stuff: Alex Clark P.O. Box 192022 Los Angeles, CA 90019 I'm ..."},
  {'vidId': 'rYfEG24vVx4',
   'publishTime': '2016-08-25T16:16:11Z',
   'vidTitle': '7 Day Vegan Challenge Baby (Solves All Your Problems)',
   'vidDescription': "Thanks Jaiden :) Now you can mail me stuff: Alex Clark P.O. Box 192022 Los Angeles, CA 90019 I'm Alex Clark and I make ..."},
  {'vidId': '2T6fRgAYsTE',
   'publishTime': '2016-06-08T12:46:56Z',
   'vidTitle': 'First Kiss Story',
   'vidDescription': "So did we go too fa

We now have the list of videoIds for each channel

In [5]:
listOfVideoIds=[]
for key, value in backend_purposes.items():
    listOfVideoIds+=value
with open('listOfVideoIds.json', 'w') as outfile:
    json.dump(listOfVideoIds, outfile)

After using this information in `main.py` to shoot at the API to get individual video metadata, I again filter out information specific to each video.

In [6]:
df = pd.read_json (r'videoListResponse.json')
matching_vidId_to_info={}
count = 0
debugMsg = []
for index, indList in df.iterrows():
    items = indList['items']
    for item in items:
        count+=1
        vidId = item['id']
        contentDetails= item['contentDetails']
        duration= contentDetails['duration']
        statistics = item['statistics']
        viewCount = statistics['viewCount']
        likeCount = statistics.get('likeCount')
        commentCount = statistics.get('commentCount')

        if (likeCount == None or commentCount == None):
            print(vidId, "has", "likes" if likeCount == None else '', "comments" if commentCount == None else '', "disabled")
        topicCategories = []
        if item.get('topicDetails') != None:
            topicCategories = item.get('topicDetails').get('topicCategories')
        if True not in ['music' in strng.lower() for strng in topicCategories] or item.get('topicDetails') == None: #all videos that are not music-typed or just don't have a type
            matching_vidId_to_info[vidId] = {'duration': duration,
                            'viewCount': int(-1 if viewCount is None else viewCount),
                            'likeCount': int(-1 if likeCount is None else likeCount),
                            'commentCount': int(-1 if commentCount is None else commentCount),
                            'topicCategories': topicCategories}
        else: #all videos that are music-typed
            debugMsg.append(vidId)
print(count)
len(matching_vidId_to_info)
print("There were ", 3208-len(matching_vidId_to_info), " videos categorized as music-videos")


QQilWUln3F8 has  comments disabled
0eCXTYobXQk has  comments disabled
hpQQohcHk9Q has  comments disabled
fV15ba-dpPI has  comments disabled
Cb6F14AGrvI has  comments disabled
OmpK2tkKmNQ has likes  disabled
3222
There were  101  videos categorized as music-videos


In [7]:
#deepcopy
channelIdToListOfVids_copy = json.loads(json.dumps(channelIdToListOfVids))
debugMsg = []
for channelId in LIST_OF_CHANNEL_IDS:
    listOfVideosForEachChannel = channelIdToListOfVids_copy[channelId]
    for video in listOfVideosForEachChannel:
        additionalVideoMetadata= matching_vidId_to_info.get(video['vidId'])
        if additionalVideoMetadata:
            video.update(additionalVideoMetadata)
        else:
            debugMsg.append(video['vidId'])

print(sum(len(channelIdToListOfVids_copy[channelId]) for channelId in LIST_OF_CHANNEL_IDS))
print(len(debugMsg))
#some of these vids are private, duplicates. 

3223
103


In [8]:
REFERENCE_UTC_TIME_FOR_PUBLISH = datetime(2021, 12, 14, 5, 2, 59, 166993)
channelId_to_earliestVideoTimestamp_and_mostViewedVideo = {} # {channelId: {firstVideoTimestamp: <>, mostViewedVideoInfo: {}}}

#channelId_to_representative_vid_and_first_vid
for channelId in LIST_OF_CHANNEL_IDS:
    listOfVideoInfo = channelIdToListOfVids_copy[channelId]
    df = pd.DataFrame(list(listOfVideoInfo))
    mostViewedVideoIndex = df['viewCount'].idxmax()
    earliestVideoTimestamp = min(df['publishTime'])
    #DO NOT RUN. IT'LL MODIFY THE _COPY
    firstUploadDate = earliestVideoTimestamp
    daysLiveSinceFirstUpload = pd.Timedelta(REFERENCE_UTC_TIME_FOR_PUBLISH - datetime.strptime(earliestVideoTimestamp, '%Y-%m-%d'+'T'+'%H:%M:%S'+'Z')).round('d')
    channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId] = {'firstUploadDate': firstUploadDate, 'daysSinceFirstVideo': daysLiveSinceFirstUpload, 'mostViewedVideo': df.iloc[mostViewedVideoIndex].to_dict()}

channelId_to_earliestVideoTimestamp_and_mostViewedVideo

{'UCsDmESjqNPukDmVnuneLrqw': {'firstUploadDate': '2009-12-07T21:13:28Z',
  'daysSinceFirstVideo': Timedelta('4389 days 00:00:00'),
  'mostViewedVideo': {'vidId': 'EcgkRp2IUsc',
   'publishTime': '2017-10-26T01:05:22Z',
   'vidTitle': 'My Hot Babysitter Got Me In Trouble',
   'vidDescription': 'First Name Babysitter. Last Name Trouble. GET T-SHIRTS: http://www.itsalexclark.com SEE ME ON TOUR: ...',
   'duration': 'PT6M30S',
   'viewCount': 35957532.0,
   'likeCount': 840531.0,
   'commentCount': 75609.0,
   'topicCategories': ['https://en.wikipedia.org/wiki/Entertainment',
    'https://en.wikipedia.org/wiki/Film']}},
 'UCY6Ij8zOds0WJEeqCLOnqOQ': {'firstUploadDate': '2015-07-18T11:43:37Z',
  'daysSinceFirstVideo': Timedelta('2341 days 00:00:00'),
  'mostViewedVideo': {'vidId': 'kbCah6yhYRs',
   'publishTime': '2019-06-11T11:00:07Z',
   'vidTitle': 'Descendants doesn&#39;t make any sense...',
   'vidDescription': "Descendants doesn't make any sense animation watch my other Disney Channel 

And now, I update the dictionary of each video specified by vidId, to update the video information with more metadata. Not only does the video info contain `vid publishedAt`, `publishTime`, `vidTitle`, and `vidDescription`, but it also contains `duration`, `viewCount`, `likeCount`, `commentCount`, and `topicCategories`

In [9]:
# with open('masterSheet.json', 'w') as outfile:
#     json.dump(channelIdToListOfVids_copy, outfile)

channelIdToListOfVidIds = {}
for channelId in LIST_OF_CHANNEL_IDS:
    vidInfoList = channelIdToListOfVids_copy[channelId]
    channelIdToListOfVidIds[channelId] = []
    for vidInfo in vidInfoList:
        channelIdToListOfVidIds[channelId].append(vidInfo['vidId'])

with open('masterSheetOfChannelIdToListOfVidIds.json', 'w') as outfile:
    json.dump(channelIdToListOfVidIds, outfile)

If I had a lot of resources like an actual GPU and memory, I'd probably analyze all the videos to find the average video to be representative of the channel, but since I don't, I want to take the most viewed video of each channel. This is a decision I made based on the... untestable hypothesis that the most-viewed video is the one that brought the channel the most subscriber count out of all of them. 

So I think taking the most-viewed video to represent the channel is an acceptable alternative. Thus, I will begin searching for 
1. the most viewed video by searching for `max(viewcount)` for each Channel
2. the first uploaded video by searching for `min(publishedAt)` for each channel because I need those stats as well. 

The result I get should be something along the lines of: `{ChannelId: 'xyz', earliestVideoPublishedDate: 'DateTime', mostViewedVideo : {vidInfo dictionary}}`

In [267]:
listOfVideoInfo = channelIdToListOfVids_copy["UCo8bcnLyZH8tBIH9V1mLgqQ"]
df = pd.DataFrame(list(listOfVideoInfo)) 
df

vidId           publishTime  \
0    CAb_bCtKuXg  2018-07-19T17:21:15Z   
1    cV542dRK3uk  2016-03-30T17:29:53Z   
2    ntHMsyr3R6w  2016-07-30T19:37:19Z   
3    2yFCyPX3kT0  2016-04-11T18:35:25Z   
4    _xBkp9fslQE  2018-03-26T19:14:47Z   
..           ...                   ...   
97   rj7Yo4AvCO8  2021-07-31T17:39:02Z   
98   0SY3yZId1sY  2021-07-17T15:41:13Z   
99   nwOB5HE8qiQ  2021-07-04T15:15:01Z   
100  SVZC5lxyz_Q  2021-06-12T17:17:58Z   
101  Ir_3Nxu6x2Y  2021-06-05T18:04:11Z   

                                              vidTitle  \
0    Life is Fun - Ft. Boyinaband (Official Music V...   
1                              Work Stories (sooubway)   
2                             My Thoughts on Roommates   
3                                   Annoying Customers   
4           My Mom&#39;s Cruel and Unusual Punishments   
..                                                 ...   
97                                     Catfish #shorts   
98                    A Villain with Disslexia #shorts   
99                                Lizard Cupid #shorts   
100                                 Dissection #shorts   
101                                  Hot Water #shorts   

                                        vidDescription duration   viewCount  \
0    just don't think about it too much Big thanks ...      NaN         NaN   
1    I was a sandwich ARTIST, okay? Because subways...  PT7M21S  78697118.0   
2    Everyone come meet my (awesome) roommates. See...  PT7M15S  88207270.0   
3    Here's part two of the subway stories. If any ...   PT7M7S  94097351.0   
4    Thanks for raising me mom! You made me like th...  PT8M10S  45394623.0   
..                                                 ...      ...         ...   
97   shorts Art by: Emilee Dummer ➤ https://www.ins...    PT14S   7277767.0   
98   shorts Art by: Emilee Dummer ➤ https://www.ins...    PT15S   4645520.0   
99   Iguana be your Valentine #shorts (yes, this ha...    PT17S   3800538.0   
100        Dissecting a frog is heartbreaking #shorts.    PT14S   6852548.0   
101  Lobster Mobster are always snappy #shorts I up...     PT7S   4016171.0   

     likeCount  commentCount  \
0          NaN           NaN   
1    1128407.0       67818.0   
2    1178687.0       90503.0   
3    1280019.0       71320.0   
4    1164849.0      149933.0   
..         ...           ...   
97    514518.0        7915.0   
98    222283.0        5104.0   
99    203222.0        4049.0   
100   467279.0        7989.0   
101   242342.0        4634.0   

                                       topicCategories  
0                                                  NaN  
1                                                   []  
2    [https://en.wikipedia.org/wiki/Lifestyle_(soci...  
3                                                   []  
4    [https://en.wikipedia.org/wiki/Entertainment, ...  
..                                                 ...  
97   [https://en.wikipedia.org/wiki/Lifestyle_(soci...  
98   [https://en.wikipedia.org/wiki/Entertainment, ...  
99   [https://en.wikipedia.org/wiki/Entertainment, ...  
100          [https://en.wikipedia.org/wiki/Knowledge]  
101  [https://en.wikipedia.org/wiki/Entertainment, ...  

[102 rows x 9 columns]

So... creating a dataframe out of the info is quite trivial. Really easy. I also just noticed that some of these youtube channels are going to have music videos... which is not in the scope of my project right now. Gonna have to remove any videos with topicCategories of music themes.

In [ ]:
REFERENCE_UTC_TIME_FOR_PUBLISH = datetime(2021, 12, 14, 5, 2, 59, 166993)
channelId_to_earliestVideoTimestamp_and_mostViewedVideo = {} # {channelId: {firstVideoTimestamp: <>, mostViewedVideoInfo: {}}}

#channelId_to_representative_vid_and_first_vid
for channelId in LIST_OF_CHANNEL_IDS:
    listOfVideoInfo = channelIdToListOfVids_copy[channelId]
    df = pd.DataFrame(list(listOfVideoInfo))
    mostViewedVideoIndex = df['viewCount'].idxmax()
    earliestVideoTimestamp = min(df['publishTime'])
    #DO NOT RUN. IT'LL MODIFY THE _COPY
    firstUploadDate = earliestVideoTimestamp
    daysLiveSinceFirstUpload = pd.Timedelta(REFERENCE_UTC_TIME_FOR_PUBLISH - datetime.strptime(earliestVideoTimestamp, '%Y-%m-%d'+'T'+'%H:%M:%S'+'Z')).round('d')
    channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId] = {'firstUploadDate': firstUploadDate, 'daysSinceFirstVideo': daysLiveSinceFirstUpload, 'mostViewedVideo': df.iloc[mostViewedVideoIndex].to_dict()}

channelId_to_earliestVideoTimestamp_and_mostViewedVideo

In [269]:
#important. date ref is 12/11 because this is when I pulled this data. 

df = pd.read_json (r'channelListResponse.json')
REFERENCE_UTC_NOW= datetime(2021, 12, 12, 5, 2, 59, 166993)
channelInfo_master_df= pd.DataFrame()

for index, row in df.iterrows():
    items=df['items'][index][0]
    snippet = items['snippet']

    title=snippet['title']
    channelId=items['id']
    publishedAt=snippet['publishedAt']
    description=snippet['description']
    viewCount=items['statistics']['viewCount']
    subscriberCount=items['statistics']['subscriberCount']
    videoCount=items['statistics']['videoCount']
    daysLive = pd.Timedelta(REFERENCE_UTC_NOW - datetime.strptime(publishedAt, '%Y-%m-%d'+'T'+'%H:%M:%S'+'Z')).round('d')
    data = {'channelName':title, 
    'channelId': channelId, 
    'daysLiveAsOfLaunch': daysLive,
    'daysSinceFirstVideo': channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId]['daysSinceFirstVideo'],
    'description': description,
    'viewCount': int(viewCount),
    'subscriberCount': int(subscriberCount),
    'videoCount': int(videoCount),
    'firstUploadDate' : channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId]['firstUploadDate'],
    'subscribersPerYearSinceFirstUpload' : float(subscriberCount/(channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId]['daysSinceFirstVideo']/365)),
    'viewCountPerYearSinceFirstUpload' : float(viewCount/(channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId]['daysSinceFirstVideo']/365)),
    }
    df_generated = pd.DataFrame(data, index=[index])
    channelInfo_master_df= channelInfo_master_df.append(df_generated)
channelInfo_master_df.head()

channelName                 channelId daysLiveAsOfLaunch  \
0        Alex Clark  UCsDmESjqNPukDmVnuneLrqw          4392 days   
1       Alex Meyers  UCY6Ij8zOds0WJEeqCLOnqOQ          2348 days   
2      WowRightMeow  UClN24S5-fc7LD0JJW0FdGWw          1946 days   
3     Andrei Terbea  UC3iNdSV_RQU7DHTGpqEW96w          3147 days   
4  Billy But Better  UC_52i6mlrzOZs_NEuSepzAg          2805 days   

  daysSinceFirstVideo                                        description  \
0           4389 days  Funny vids about my life.  I'm thankful for th...   
1           2341 days  I make cartoons and sometimes video essays abo...   
2           1809 days  Yo! I'm Jamie, a dorky human from Wales. Did a...   
3           3149 days  Doing animated commentary and solving crimes w...   
4           1736 days  Yes, it's a foot.\n\nSocial Medias:\nTwitter -...   

   viewCount  subscriberCount  videoCount       firstUploadDate  \
0  471903125          4280000         317  2009-12-07T21:13:28Z   
1  472627239          2870000         256  2015-07-18T11:43:37Z   
2   29732357           537000          33  2016-12-31T15:08:07Z   
3  292193694          2860000         136  2013-04-30T20:01:32Z   
4   12166120           260000          23  2017-03-14T02:31:14Z   

   subscribersPerYearSinceFirstUpload  viewCountPerYearSinceFirstUpload  
0                        4.119621e-09                      4.542201e-07  
1                        5.179163e-09                      8.528967e-07  
2                        1.254050e-09                      6.943363e-08  
3                        3.836829e-09                      3.919921e-07  
4                        6.327072e-10                      2.960612e-08

`MostViewedVideoInfo` is showing up as NaN because it's of type `dict`. So need to just make sure I have access to the deets.

In [272]:
mostViewedVideo_master_dict = {}
for channelId in LIST_OF_CHANNEL_IDS:
    data = channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId]['mostViewedVideo']
    mostViewedVideo_master_dict[channelId]= data
    
mostViewedVideo_master_df = pd.DataFrame.from_dict(mostViewedVideo_master_dict).T
mostViewedVideo_master_df

vidId           publishTime  \
UCsDmESjqNPukDmVnuneLrqw  EcgkRp2IUsc  2017-10-26T01:05:22Z   
UCY6Ij8zOds0WJEeqCLOnqOQ  kbCah6yhYRs  2019-06-11T11:00:07Z   
UClN24S5-fc7LD0JJW0FdGWw  ewsGmhAjjjI  2018-10-09T20:50:50Z   
UC3iNdSV_RQU7DHTGpqEW96w  gA0bi-bFEYs  2018-06-22T19:20:36Z   
UC_52i6mlrzOZs_NEuSepzAg  so1_5hYUEE8  2017-09-20T22:12:54Z   
UCr3cBLTYmIK9kY0F_OdFWFQ  xa-4IAR_9Yw  2016-03-29T09:20:12Z   
UCV6g95OBbVtFmN9uiJzkFqQ  I9uWUw1fxOY  2020-10-23T15:59:47Z   
UCgCKYs56-LKEPGQ99DzqQOg  W8P5ewPk9fM  2019-01-18T20:00:06Z   
UCOwxx9VnEnlFKt5EB70KTzQ  12Ne9n40tmw  2017-02-25T19:46:48Z   
UC2Ds30pkifFVD0CE08wF50g  Mv8OkBjySGQ  2017-09-15T00:08:53Z   
UC7UGbBVrqLlq6CRxWWUmyKw  2juKkLxdQo0  2019-05-11T17:14:40Z   
UCn1XB-jvmd9fXMzhiA6IR0w  eNGgfPs0Xp8  2016-09-26T21:49:18Z   
UCFeqAfEuKm7lIg2ddQzh61A  rlSXaDq3uOk  2019-06-29T18:02:32Z   
UCAlxwHEm1bWWYoe5VCSQYEg  Vm6Yu2N-ePI  2017-05-03T20:25:20Z   
UCz7iJPVTBGX6DNO1RNI2Fcg  0Vxp_Lj2b-E  2015-05-31T21:33:29Z   
UCOAUz4d1NUNxSHh_SOJtdGQ  5pMckBGWzAY  2020-05-17T19:01:04Z   
UCGGTAB19HlHEWPwwmxHsEKA  de8PRd_d7kg  2019-03-14T18:00:01Z   
UCOsATJw-IZgqGT8MFrHjKGg  oKLbOxLJfRg  2019-04-21T22:46:53Z   
UCsKVP_4zQ877TEiH_Ih5yDQ  Y7lYeRqhQ9Q  2018-09-07T18:56:30Z   
UCOinp9hALPUcZeoL3Kmaviw  BErOLQBZ6c8  2018-05-25T18:47:23Z   
UCGwu0nbY2wSkW8N-cghnLpA  plSyrHqUh78  2019-11-10T19:03:17Z   
UCu6v4AdYxVH5fhfq9mi5llA  o0zjRGRYEhk  2019-03-08T15:00:10Z   
UCj74rJ9Lgl3WTngq675wxKg  hpQQohcHk9Q  2019-08-03T15:15:01Z   
UCNIuvl7V8zACPpTmmNIqP2A  _uk_6vfqwTA  2018-03-15T18:52:13Z   
UCo8bcnLyZH8tBIH9V1mLgqQ  2yFCyPX3kT0  2016-04-11T18:35:25Z   
UCR0O-1cvuPNxDosvSDLpWHg  vuk2NZ0YKAE  2019-10-18T21:00:19Z   
UCvUmwreRrbxeR1mbmojj8fg  cZ_CnLE6SPo  2017-11-21T19:03:12Z   
UCUKi4zY5ETSqrKAjTBgjM-g  7FnQrNFyWy8  2014-08-11T08:49:01Z   
UCDQBZcjYKP1J1Nu-Y0_D37Q  F8A-tXp09fs  2018-09-01T17:20:01Z   
UCQ9HvHH-KRYHI5ynj2kbLwQ  7WSo1Uw-p_g  2017-08-20T21:50:08Z   
UCRfg0SWjIHm_h95e4V8X5og  PgajWuZA408  2017-11-20T18:12:39Z   
UCWPB0WpnMIy-g7zncwIhvQg  A6V1QujNz8s  2019-01-25T21:30:02Z   
UCC-slLJZ4p4HOznMUcFn_2g  LdNi3PpGtl8  2017-12-09T19:21:33Z   

                                                                   vidTitle  \
UCsDmESjqNPukDmVnuneLrqw                My Hot Babysitter Got Me In Trouble   
UCY6Ij8zOds0WJEeqCLOnqOQ          Descendants doesn&#39;t make any sense...   
UClN24S5-fc7LD0JJW0FdGWw  I have APHANTASIA (and you may too...without r...   
UC3iNdSV_RQU7DHTGpqEW96w              How KSI Saved Logan Paul&#39;s Career   
UC_52i6mlrzOZs_NEuSepzAg                Being a Boba Barista (Work Stories)   
UCr3cBLTYmIK9kY0F_OdFWFQ               Casually Explained: Is She Into You?   
UCV6g95OBbVtFmN9uiJzkFqQ                          The Life of a Speedrunner   
UCgCKYs56-LKEPGQ99DzqQOg          Why I Dont Want a Hot Wife (Ft. Emirichu)   
UCOwxx9VnEnlFKt5EB70KTzQ  WHO&#39;S YOUR DADDY? ANIMATED with Pewdiepie ...   
UC2Ds30pkifFVD0CE08wF50g           My Super HOT English Teacher (Animation)   
UC7UGbBVrqLlq6CRxWWUmyKw                   By the way, Are You a Sociopath?   
UCn1XB-jvmd9fXMzhiA6IR0w                                            Crushes   
UCFeqAfEuKm7lIg2ddQzh61A                                Childhood Crushes 2   
UCAlxwHEm1bWWYoe5VCSQYEg                     Weird Kids I met in Art School   
UCz7iJPVTBGX6DNO1RNI2Fcg                 Things I Found Stupid About School   
UCOAUz4d1NUNxSHh_SOJtdGQ             I Played Animal Crossing for 300 Hours   
UCGGTAB19HlHEWPwwmxHsEKA                                        Pop Culture   
UCOsATJw-IZgqGT8MFrHjKGg                                              hurt.   
UCsKVP_4zQ877TEiH_Ih5yDQ                How I Met My Abusive (ex) Boyfriend   
UCOinp9hALPUcZeoL3Kmaviw       ARTIST PROBLEMS (Ft. TheOdd1sOut and Tabbes)   
UCGwu0nbY2wSkW8N-cghnLpA              I Attempted my First Pokemon Nuzlocke   
UCu6v4AdYxVH5fhfq9mi5llA                              My School Dress Codes   
UCj74rJ9Lgl3WTngq675wxKg                     &quot;just buy a new one&qu

In [271]:
flatList = sum([mostViewedVideo_master_df.loc[channelId]['topicCategories'] for channelId in LIST_OF_CHANNEL_IDS], []) #monoid. apparently pretty bad performance
set(flatList)

{'https://en.wikipedia.org/wiki/Entertainment',
 'https://en.wikipedia.org/wiki/Film',
 'https://en.wikipedia.org/wiki/Hobby',
 'https://en.wikipedia.org/wiki/Knowledge',
 'https://en.wikipedia.org/wiki/Lifestyle_(sociology)',
 'https://en.wikipedia.org/wiki/Pet',
 'https://en.wikipedia.org/wiki/Role-playing_video_game',
 'https://en.wikipedia.org/wiki/Simulation_video_game',
 'https://en.wikipedia.org/wiki/Society',
 'https://en.wikipedia.org/wiki/Video_game_culture'}

So I need to remove... all the music-themed stuff. And I did. All the way up there.

Whew. And now... I need to gather the transcripts for the 33 videos above to run semantic analysis on it. There's probably an API for it...

In [278]:
channelId_to_mostViewedVidId = {}

for channelId in LIST_OF_CHANNEL_IDS:
    mostViewedVidId = mostViewedVideo_master_dict[channelId]['vidId']
    channelId_to_mostViewedVidId[channelId] = mostViewedVidId

with open('channelId_to_mostViewedVidId.json', 'w') as outfile:
    json.dump(channelId_to_mostViewedVidId, outfile)

In [ ]:
zip()